In [1]:
pip install datasets scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [3]:
# Try loading a specific split of the dataset
dataset = load_dataset("dair-ai/emotion")

# Check the loaded data
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [4]:
# Memeriksa label yang tersedia dalam dataset
train_data = dataset['train']
print(train_data.features['label'].names)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Menggabungkan data latih dan validasi
data = dataset['train']
texts = data['text']
labels = data['label']  # Menggunakan 'labels' untuk dataset multilabel

# TF-IDF vektorizer
max_features = 10000  # Jumlah kata unik yang akan digunakan
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
x_data = tfidf_vectorizer.fit_transform(texts).toarray()

# Mengubah label menjadi bentuk yang sesuai
labels = [label if isinstance(label, list) else [label] for label in labels]

# Binarisasi label
mlb = MultiLabelBinarizer()
y_data = mlb.fit_transform(labels)

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
classification_reports = []
fold = 1

class_names = [str(label) for label in range(y_data.shape[1])]  # assuming class names are just the class labels

for train_index, val_index in kf.split(x_data):
    x_train, x_val = x_data[train_index], x_data[val_index]
    y_train, y_val = y_data[train_index], y_data[val_index]
    
    # Membuat model Random Forest
    model = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
    model.fit(x_train, y_train)
    
    # Prediksi dan evaluasi
    y_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)
    report = classification_report(y_val, y_pred, target_names=class_names, zero_division=0)
    classification_reports.append(report)
    print(f'Fold {fold} accuracy: {accuracy:.4f}')
    print(f'Classification Report for Fold {fold}:')
    print(report)
    print('---------------------------------------------------------')
    
    fold += 1

# Rata-rata akurasi
print(f'Mean accuracy: {np.mean(accuracies):.4f}')

# Classification report untuk data validasi terakhir
final_y_pred = model.predict(x_val)
final_report = classification_report(y_val, final_y_pred, target_names=class_names, zero_division=0)

print('---------------------------------------------------------')
print('Classification Report for Final Validation Set:')
print(final_report)

Fold 1 accuracy: 0.7478
Classification Report for Fold 1:
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       946
           1       0.92      0.82      0.86      1021
           2       0.87      0.60      0.71       296
           3       0.89      0.84      0.86       427
           4       0.91      0.75      0.82       397
           5       0.96      0.57      0.71       113

   micro avg       0.91      0.80      0.85      3200
   macro avg       0.91      0.74      0.81      3200
weighted avg       0.91      0.80      0.85      3200
 samples avg       0.77      0.80      0.78      3200

---------------------------------------------------------
Fold 2 accuracy: 0.7394
Classification Report for Fold 2:
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       932
           1       0.93      0.80      0.86      1106
           2       0.90      0.60      0.72       281
           3